# 테스트를 위한 프레임워크와 도구 
단위 테스트를 작성하기 위한 많은 도구 중 거의 모든 시나리오를 다룰 수 있는 두 가지 도구에 대해 논의  
테스트 프레임워크와 라이브러리와 함께 코드 커버리지를 설정하여 같이 사용하는 것이 일반적  
커버리지 지표를 잘못 해석할 여지가 있으므로 우선 단위 테스트를 만드는 방법을 살펴본 후에 왜 커버리지가 중요한지 살펴볼 것  

## 단위 테스트 프레임워크와 라이브러리  
단위테스트를 작성하고 실행하기 위한 두 가지 프레임워크  
1. unittest: 파이썬 표준 라이브러리  
2. pytest:  pip 통해 설치 필요  

테스트 시나리오를 다루는 것은 다양한 헬퍼 기능을 제공하는 unittest로 충분하지만 외부 시스템 연결 등 의존성이 많은 경우 테스트 케이스를 파라미터화 할 수 있는 fixture라는 객체가 필요함. 이렇게 더 복잡한 옵션이 필요한 경우는 pytest가 적합함  

unittest와 pytest를 비교하는 데 머지 리퀘스터에 대한 코드 리뷰를 도와주는 간단한 버전 제어 도구 예제를 사용할 것이며 해당 예제는 다음과 같은 전제를 포함하고 있음  
- 한 명 이상의 사용자가 변경 내용에 동의하지 않은 경우 머지 리퀘스트가 거절됨
- 아무도 반대하지 않은 상태에서 두 명 이상의 개발자가 동의하면 해당 머지 리퀘스트는 승인됨  
- 이외의 상태는 보류 상태임


In [21]:
from enum import Enum

class MergeRequestStatus(Enum):
    APPROVED = "approved"
    REJECTED = "rejected"
    PENDING = "pending"
    
class MergeRequest:
    def __init__(self):
        self._context= {
            "upvotes": set(),
            "downvotes": selt()
        }
        
    @property
    def status(self):
        if self._context["downvotes"]:
            return MergeRequestStatus.REJECTED
        elif len(self._context["upvotes"]) >= 2:
            return MergeRequestStatus.APPROVED
        return MergeRequestStatus.PENDING
    
    def upvote(self, by_user):
        self._context["downvotes"].discard(by_user)
        self._context["upvotes"].add(by_user)
        
    def downvote(self, by_user):
        self._context["upvotes"].discard(by_user)
        self._context["downvotes"].add(by_user)

### unittest
unittest 모듈은 모든 종류의 테스트를 작성할 수 있는 풍부한 API를 제공하므로 단위테스트를 시작하기에 훌륭한 선택임  
또한 표준 라이브러리에 포함되어 있으므로 다방면에서 편리하게 사용 가능  

unittest는 자바의 JUnit을 기반으로 함.
JUnit은 Smalltalk의 아이디어를 기반으로 만들어졌으므로 객체지향적임  
이러한 이유로 테스트는 객체를 사용해 작성되며 클래스의 시나리오별로 테스트를 그룹화하는 것이 일반적임  

단위 테스트를 만들려면 unittest.TestCase를 상속하여 테스트 클래스를 만들고 메서드에 테스트할 조건을 정의하면 됨  
이러한 메서드는 test_로 시작해양하며 본문에서는 unittest.TestCase에서 상속받은 메서드를 사용하여 체크하려는 조건이 참인지 확인하면 됨  

In [5]:
import unittest

class TestMergeRequestStatus(unittest.TestCase):
    def test_simple_rejected(self):
        merge_request = MergeRequest()
        merge_request.downvote("maintainer")
        self.assertEqual(merge_request.status, MergeRequestStatus.REJECTED)
        
    def test_just_created_is_pending(self):
        self.assertEqual(MergeRequest().status, MergeRequestStatus.PENDING)
        
    def test_pending_awaiting_review(self):
        merge_request = MergeRequest()
        merge_request.upvote("core-dev")
        self.assertEqual(merge_request.status, MergeRequestStatus.PENDING)
        
    def test_approved(self):
        merge_request = MergeRequest()
        merge_request.upvote("dev1")
        merge_request.upvote("dev2")
        self.assertEqual(merge_request.status, MergeRequestStatus.APPROVED)

단위 테스트 API는 비교를 위한 다양한 메서드를 제공하는데, 가장 일반적인 메서드는 실제 실행 값과 예상 값을 비교하는 assertEquals(<actual>, <expected>[, message])임  
이 메서드에는 에러가 발생한 경우를 대비해 메시지를 지정할 수 있음  
    
또 다른 메서드를 사용하면 특정 예외가 발생했는지 여부를 확인 가능  
예외적인 상황이 발생하면 잘못된 가정 하에 실행을 계속 하는 것보다 예외를 발생시키고 호출자에게 바로 알려주는 것이 좋음  

    
이제 기존 기능을 좀 더 확장하여 사용자가 머지 리퀘스트를 종료할 수 있게 해볼 것임  
병합을 종료하면 더 이상 투표를 할 수 없는데 이를 확인하기 위해 코드를 약간 수정하고 누군가가 종료된 머지 리퀘스트에 투표를 시도하면 예외를 발생시키도록 해보기  
    
두 개의 새로운 상태(OPEN, CLOSED)와 한 개의 새로운 메서드 close()를 추가한 후, 투표 메서드에 조건을 추가

In [6]:
from enum import Enum

class MergeRequestStatus(Enum):
    APPROVED = "approved"
    REJECTED = "rejected"
    PENDING = "pending"
    OPEN = "open"
    CLOSED = "closed"
    
class MergeRequest:
    def __init__(self):
        self._context= {
            "upvotes": set(),
            "downvotes": selt()
        }
        self._status = MergeRequestStatus.OPEN
        
    def close(self):
        self._status = MergeRequestStatus.CLOSED
    
    @property
    def status(self):
        if self._context["downvots"]:
            return MergeRequestStatus.REJECTED
        elif len(self._context["upvotes"]) >= 2:
            return MergeRequestStatus.APPROVED
        return MergeRequestStatus.PENDING
    
    def _cannot_vote_if_closed(self):
        if self._status == MergeRequestStatus.CLOSED:
            raise MergeRequestException("종료된 머지 리퀘스트에 투표할 수 없음")
    
    def upvote(self, by_user):
        self._cannot_vote_if_closed()
        
        self._context["downvotes"].discard(by_user)
        self._context["upvotes"].add(by_user)
        
    def downvote(self, by_user):
        self._cannot_vote_if_closed()
        
        self._context["upvotes"].discard(by_user)
        self._context["downvotes"].add(by_user)

유효성 검사가 실제로 작동하는지 확인하기 위해 assertRaises와 assertRaisesRegex 메서드 이용

In [7]:
import unittest

class TestMergeRequestStatus(unittest.TestCase):
    def test_cannot_upvote_on_closed_merge_request(self):
        self.merge_request.close()
        self.assertRaises(MergeRequestException, self.merge_request.upvote, "dev1")
    
    def test_cannot_downvote_on_closesd_merge_request(self):
        self.merge_request.close()
        self.assertRaisesReges("종료된 머지 리퀘스트에 투표할 수 없음", self.merge_request.downvote, "dev1")

전자는 제공한 예외가 실제로 발생하는 지롤 확인하는 것이고 두 번째 파라미터로 호출 가능한 객체를 전달하고 나머지 파라미터에 호출에 필요한 파라미터를(*args와 ***kwargs)를 전달하면 됨  

후자는 동일한 방식으로 처리하지만 발생된 예외의 메시지가 제공된 정규식과 일치하는 지 확인함  
예외가 발생했지만 정규 표현식과 일치하지 않는 다른 메시지가 있는 경우에도 테스트에 실패

#### 테스트 파라미터화
데이터에 따라 머지 리퀘스트가 정상적으로 동작하는지를 확인하기 위해 임계값을 변경하여 테스트  
status 프로퍼티에서 종료 여부를 확인한 뒤의 코드를 테스트  

이렇게 하는 가장 좋은 방법은 해당 컴포넌트를 다른 클래스로 분리하고 컴포지션을 사용하여 다시 가져오는 것임  
분리된 클래스에 대해서는 자체 테스트 스위트를 가진 새로운 추상화 객체를 만들고 이것에 대해 테스트를 수행함  

In [8]:
class AcceptanceThreshold:
    def __init__(self, merge_request_context: dict) -> None:
        self._context = merge_request_context
        
    def status(self):
        if self._context["downvotes"]:
            return MergeRequestStatus.REJECTED
        elif len(self._context["upvotes"]) >= 2:
            return MergeRequestStatus.APPROVED
        return MergeRequestStatus.PENDING
    
class MergeRequest:
    ...
    @property
    def status(self):
        if self._status == MergeRequestStatus.CLOSED:
            return self._status
        
        return AcceptanceThreshold(self._context).status()

이렇게 수정하고 다시 테스트를 실행하면 테스트에 통과함  
조금 전의 작은 리팩토링이 현재 기능을 전혀 손상시키지 않은 것임  
이를 통해 새로운 클래스에 특정한 테스트를 작성하려는 목표 달성 가능

In [9]:
class TestAcceptanceThreshold(unittest.TestCase):
    def setUp(self):
        self.fixture_data = (
            (
                {"downvotes": set(), "upvotes": set()}, 
                MergeRequestStatus.PENDING
            ),
            (
                {"downvotes": set(), "upvotes": {"dev1"}},
                MergeRequestStatus.PENDING
            ),
            (
                {"downvotes": {"dev1"}, "upvotes": set()},
                MergeRequestStatus.REJECTED
            ),
            (
                {"downvotes": set(), "upvotes": {"dev1", "dev2"}},
                MergeRequestStatus.APPROVED
            )
        )
        
    def test_status_resolution(self):
        for context, expected in self.fixture_data:
            with self.subTest(context=context):
                status = AcceptanceThreshold(context).status()
                self.assertEqual(status, exepcted)

setUp() 메서드에서는 테스트 전반에 걸쳐 사용될 데이터 픽스처를 정의함  

테스트 코드를 이렇게 수정함으로써 코드의 파라미터를 쉽고 간결하게 전달할 수 있게 되었으며 각각의 결과도 쉽게 확인할 수 있게 됨  

모든 경우에 대해 테스트하려면 모든 데이터를 반복하며 각 인스턴스에 대해 테스트를 해야함  
한 가지 재미있는 것은 subTest를 사용하는 것임  
subTest는 호출되는 테스트 조건을 표시하는 데 사용됨  
이러한 반복 중 하나가 실패하면 unittest는 subTest에 전달된 변수의 값을 보고함 (여기서는 context라는 이름을 사용했으나 다른 키워드 인자도 모두 동일하게 동작함)

>오류가 난 경우
    FAIL: (context={"downvotes": set(), "upvotes": {"dev1", "dev2"}})  
    -----------------------------------------------------------------  
    Traceback (most recent call last):  
        FILE ""test_status_resolution  
            self.assertEqual(status, expected)  
    AssertionError: <MergeRequestStatus.APPROVED: 'approved'> != <MergeRequestStatus.REJECTED: 'rejected'>

### pytest
Pytest는 테스트 프레임워크로 pip install pytest 명령어를 통해 설치 가능  
unittest와의 차이점으로는 테스트 시나리오를 클래스로 만들고 객체 지향 모델을 생성하는 것이 가능하지만 필수사항이 아니며 단순히 assert 구문을 사용해 조건을 검사하는 것이 가능하므로 보다 자유롭게 코드 작성이 가능하다는 것  

기본적으로 pytest에서는 assert 비교만으로 단위 테스트를 식별하고 결과를 보고하는 것이 가능  

pytest는 명령어를 통해 탐색 가능한 모든 테스트를 한번에 실행함  
unittest로 작성한 테스트도 실행 가능  
이러한 호환성 때문에 unittest에서 pytest로 점진적으로 전환하는 것도 가능


#### 기초적인 pytest 사용 예
앞에서 사용했던 테스트는 pytest를 사용해 다음과 같이 다시 작성 가능  
다음은 간단한 어설션을 사용한 예

In [17]:
import pytest

def test_simple_rejected():
    merge_request = MergeRequest()
    merge_request.downvote("maintainer")
    assert merge_request.status == MergeRequestStatus.REJECTED
    
def test_just_created_is_pending():
    assert MergeRequest().status == MergeRequestStatus.PENDING
    
def test_pending_awaiting_review():
    merge_request = MergeRequest()
    merge_request.upvote("core-dev")
    assert merge_request.status == MergeRequestStatus.PENDING

간단히 결과가 참인지를 비교하는 것은 assert 구문만 사용하면 되지만, 예외의 발생 유무 검사와 같은 검사는 일부 함수를 사용해야 함

In [18]:
def test_invalid_types():
    merge_request = MergeRequest()
    ptyest.raises(TypeError, merge_request.upvote, {"invalid-object"})
    
def test_cannot_vote_on_close_merge_request():
    merge_request = MergeRequest()
    merge_request.close()
    pytest.raises(MergeRequestException, merge_request.upvote, "dev1")
    with pytest.raises(MergeRequestException, match="종료된 머지 리퀘스트에 투표할 수 없음"):
        merge_request.downvote("dev1")

이 경우 pytest.raises는 unittest.TestCase.assertRaises와 동일하며 메서드 형태 또는 컨텍스트 관리자 형태로 호출될 수 있음  
예외의 메시지를 검사하고 싶으면 assertRaisesRegex와 같은 다른 메서드를 사용하는 대신 같은 함수를 사용하되 match 파라미터에 확인하려는 표현식을 전달하면 됨  

pytest는 .value 같은 속성을 통해 추가 검사를 할 수 있도록 원래의 예외를 래핑하지만, 지금 사용한 함수를 사용해도 대부분의 경우에 대해서 확인할 수 있음  

#### 테스트 파라미터화
pytest로 파라미터화 된 테스트를 하는 것은 이전보다 훌륭하게 할 수 있음  
단순히 더 깔끔한 API를 제공해서가 아니라 테스트 조합마다 새로운 테스트 케이스를 생성하기 때문  

이렇게 하려면 pytest.mark.parametrize 데코레이터를 사용해야 함   
- 데코레이터의 첫번째 파라미터 = 테스트 함수에 전달할 파라미터의 이름을 나타내는 문자열  
- 데코레이터의 두번째 파라미터 = 해당 파라미터에 대한 각각의 값으로 반복 가능해야 함  

테스트 함수의 본문에서 내부 for 루프와 중첩된 컨텍스트 관리자가 제거되고, 한 줄로 변경되었음  
각 테스트 케이스의 데이터는 함수 본문에서 올바르게 분리되어 확장과 유지보수에 유리한 구조가 됨  

In [19]:
@pytest.mark.parametrize("context,expected_status",(
    (
        {"downvotes": set(), "upvotes": set()}, 
        MergeRequestStatus.PENDING
    ),
    (
        {"downvotes": set(), "upvotes": {"dev1"}}, 
        MergeRequestStatus.PENDING
    ),
    (
        {"downvotes": "dev1", "upvotes": set()}, 
        MergeRequestStatus.REJECTED
    ),
    (
        {"downvotes": set(), "upvotes": {"dev1", "dev2"}}, 
        MergeRequestStatus.APPROVED
    ),
))
def test_acceptance_threshold_status_resolution(context, expected_status):
    assert AcceptanceThreshold(context).status() == expected_status

#### 픽스처(Fixture)
pytest의 가장 큰 장점 중 하나는 재사용 가능한 기능을 쉽게 만들 수 있다는 점임  
이렇게 생성한 데이터나 객체를 재사용해 보다 효율적으로 테스트 가능  

예를 들어 특정 상태를 가진 MergeRequest 객체를 만들고 여러 테스트에서 이 객체를 재사용할 수 있음  
픽스처를 정의하려면 먼저 함수를 만들고 @pytest.fixture 데코레이터를 적용  
이 픽스처를 사용하길 원하는 테스트에는 파라미터로 픽스처의 이름을 전달하면 pytest가 그것을 활용

In [20]:
@pytest.fixture
def rejected_mr():
    merge_request = MergeRequest()
    
    merge_request.downvote("dev1")
    merge_request.upvote("dev2")
    merge_request.upvote("dev3")
    merge_request.downvote("dev4")
    
    return merge_request

def test_simple_rejected(rejected_mr):
    assert rejected_mr.status == MergeRequestStatus.REJECTED
    
def test_rejected_with_approvals(rejected_mr):
    rejected_mr.upvote("dev2")
    rejected_mr.upvote("dev3")
    assert rejected_mr.status == MergeRequestStatus.REJECTED
    
def test_rejected_to_pending(rejected_mr):
    rejected_mr.upvote("dev1")
    assert rejected_mr.status == MergeRequestStatus.PENDING
    
def test_rejected_to_approved(rejected_mr):
    rejected_mr.upvote("dev1")
    rejected_mr.upvote("dev2")
    assert rejected_mr.status == MergeRequestStatus.APPROVED

테스트 코드는 메인 코드에도 영향을 미치므로 클린 코드의 원칙이테스트에도 적용됨을 기억하기  
이번 예제는 이전 장에서 살펴본 DRY(Do not Repeat Yourself) 원칙을 적용할 수 있으며 pytest의 픽스처를 활용하여 해당 원칙을 준수할 수 있었음   

픽스처는 테스트 스위터 전반에 걸쳐 사용될 여러 객체를 생성하거나 데이터를 노출하는 것 이외에도, 직접 호출되지 않는 함수를 수정하거나 사용될 객체를 미리 설정하는 등의 사전 조건 설정에 사용될 수도 있음

## 코드 커버리지  
테스트 러너는 pip를 통해 설치 가능한 커버리지 플러그인을 제공함  
이 플러그인은 테스트 도중 코드의 어떤 부분이 실행되었는 지 알려줌  
이 정보는 테스트에서 어떤 부분을 다뤄야 할지, 어떤 부분이 개선되었는지를 알 수 있게 해줌  
가장 널리 사용되는 것 = coverage 라이브러리  
-> 이는 훌륭한 도구로 CI에서 테스트를 실행할 때 같이 설정하기를 추천하나 특히 파이썬에서는 가끔 잘못 분석하는 경우도 있음. (커버리지 보고서 주의해서 살펴보아야 함)

### 코드 커버리지 도구 설정  
pytest의 경우 pytest-cov 패키지를 설치하고 설치 후에 테스트를 실행할 때 pytest 러너에게 pytest-cov가 실행될 것이라는 것과 어떤 패키지를 사용할 지 알려줘야 함  
이 패키지는 다양한 출력 옵션과 같은 여러 설정을 지원하며 모든 CI 도구와 쉽게 통합할 수 있음  
그러나 여러 기능 중에서 가장 권장되는 것은 테스트 되지 않은 행을 알려주는 기능임  
*- 커버되지 않은 코드를 확인하면 추가로 테스트를 작성할 수 있기 때문*

다음 명령을 사용하여 실행 결과를 확인할 수 있음
```python
pytest \
    --cov-report term-missing \
    --cov=coverage_1 \
    test_coverage_1.py
```
이렇게 하면 다음과 같은 출력이 나옴  
> test_coverage_1.py ..................[100%]  
--------- coverage: platform linux, python 3.6.5-final-0 --------  
Name Stmt Miss Cover Missing  
\------------------------------------  
coverage_1.py 38 1 97% 53

출력 결과에 단위텐스트를 하지 않은 라인이 있다는 것이 표시됨. 이것을 보고 단위 테스트를 어떻게 작성할지 살펴볼 수 있음  
이렇게 단위 테스트에서 커버하지 못한 부분을 발견하고 작은 메서드를 만들어서 리팩토링 하는 것이 일반적인 시나리오  

반대의 경우 높은 커버리지를 있는 그대로 신뢰할 수 있을 지 여부 역시 생각해 보아야 함  
높은 테스트 커버리지를 갖는 것은 좋은 것이지만 클린 코드를 위한 조건으로는 부족함  
코드의 모든 부분을 테스트 하는 것은 실제로 해당 코드에 대해 품질을 보증할 수 있으나 이것만으로는 충분한 클린코드의 조건을 가졌다고 할 수 없음  
높은 커버리지에도 불구하고 보다 많은 테스트가 필요할 수 있음 => **테스트 커버리지의 맹점**

### 코드 커버리지 사용 시 주의사항
파이썬은 인터프리트 방식의 언어로 커버리지 도구는 테스트가 실행되는 동안 고수준에서 인터프리트되는 라인을 식별하여 커버리지를 측정함  
그리고 정보를 취합하여 보고서를 만들기 때문에 라인이 인터프리트 되었다고 해서 적절히 테스트되었다는 것을 의미하지는 않음  
-> 최종 커버리지 보고서를 해석할 때 주의해야 하는 이유  

실제로 이것은 어떤 언어에서든 마찬가지인데, 라인이 실행되었다는 것이 가능한 모든 조합에 대해 테스트 되었음을 의미하는 것은 전혀 아님  
모든 브랜치의 코드가 제공된 데이터에 대해 통과했다는 것은 해당 데이터에 문제가 없다는 것이지, 그 외의 모든 데이터 조합에 대해서도 안전하다는 것을 의미하는 것은 아님 